In [1]:
%load_ext autoreload
%autoreload 2

In [157]:
from torch.utils.data import TensorDataset, DataLoader



train_dataset = TensorDataset(torch.from_numpy(train_data).to(torch.float), torch.from_numpy(train_data.reshape(100, -1)[:,40:]).to(torch.float))
train_loader = DataLoader(train_dataset, batch_size=min(32, len(train_dataset)), shuffle=True, drop_last=True)
train_dataset

for batch in train_loader:
    print(batch[0].shape)
    print(batch[1].shape)
    break

torch.Size([32, 96, 1])
torch.Size([32, 56])


In [192]:
from matplotlib import pyplot as plt
from models.losses import *
from ts2vec import TS2Vec
import pandas as pd
import numpy as np
import datautils
import torch

# # Load the ECG200 dataset from UCR archive
train_data, train_labels, test_data, test_labels = datautils.load_ECG()
# # (Both train_data and test_data have a shape of n_instances x n_timestamps x n_features)

# Train a TS2Vec model
model = TS2Vec(
    input_dims=1,
    device=0,
    output_dims=320
)
loss_log = model.fit(
    train_data,
    expert_features=train_data.reshape(100, -1)[:,40:],
    verbose=True
)

# Compute timestamp-level representations for test set
test_repr_tl = model.encode(test_data)  # n_instances x n_timestamps x output_dims

# Compute instance-level representations for test set
test_repr_il = model.encode(test_data, encoding_window='full_series')  # n_instances x output_dims

# Sliding inference for test set
test_repr_si = model.encode(
    test_data,
    casual=True,
    sliding_length=1,
    sliding_padding=50
)  # n_instances x n_timestamps x output_dims
# (The timestamp t's representation vector is computed using the observations located in [t-50, t])

tensor(-7.5586, grad_fn=<MulBackward0>) tensor(-7.5590, grad_fn=<MulBackward0>)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of ExpBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [202]:
# pd.DataFrame(test_repr_il)[4].plot()

In [43]:
batch_xi     = torch.randn(32, 50)
batch_xj     = torch.randn(32, 50)
batch_f      = torch.randn(32, 10)


In [203]:

quadratic_contrastive_loss(batch_xi, batch_f)

tensor(0.0015)

In [204]:
expclr_loss(batch_xi, batch_f)

tensor(-0.3593)

In [63]:
from tensorflow.keras.layers import LSTM
import tensorflow as tf

inputs = tf.random.normal([5, 3, 10])
lstm = tf.keras.layers.LSTM(20)
output = lstm(inputs)
print(output.shape)


(5, 20)


In [90]:
rnn = nn.LSTM(10, 20, batch_first=True)
input = torch.randn(5, 3, 10)
# h0 = torch.randn(2, 3, 20)
# c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input)

In [184]:
hn.reshape(hn.shape[1:]).shape

torch.Size([5, 20])

In [91]:
print( output.shape )
print( hn.shape )
print( cn.shape )

torch.Size([5, 3, 20])
torch.Size([1, 5, 20])
torch.Size([1, 5, 20])


In [189]:
hn.squeeze(0)

torch.Size([5, 20])

In [181]:
output[:,-1:,:].reshape(5, 20).shape

torch.Size([5, 20])

In [120]:
output[:,-1:,:].reshape(5, 20) == hn.reshape(5, 20)

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True]])

In [7]:
# data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = datautils.load_forecast_csv("ETTh2")
# train_data = data[:, train_slice]
        
# # Train a TS2Vec model
# model = TS2Vec(
#     input_dims=1,
#     device=0,
#     output_dims=320
# )
# loss_log = model.fit(
#     train_data,
#     verbose=True
# )

# # Compute timestamp-level representations for test set
# test_repr = model.encode(test_data)  # n_instances x n_timestamps x output_dims

# # Compute instance-level representations for test set
# test_repr = model.encode(test_data, encoding_window='full_series')  # n_instances x output_dims

# # Sliding inference for test set
# test_repr = model.encode(
#     test_data,
#     casual=True,
#     sliding_length=1,
#     sliding_padding=50
# )  # n_instances x n_timestamps x output_dims
# # (The timestamp t's representation vector is computed using the observations located in [t-50, t])